In [10]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
from nltk import ngrams

import re
import numpy as np
import math

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Text Processing


In [3]:
!ls
%cd drive/
!ls
%cd "My Drive"
# !ls
%cd assigmment_nlp
!ls

with open("speeches.txt", 'r', encoding='utf-8') as myfile:
  c = myfile.read()
  c=c.lower()
  # print(c)
  token = sent_tokenize(c)
  token = [re.sub(r'[^a-z0-9]'," ", i) for i in token]
  # fdist = nltk.FreqDist(token)
# fdist

token = ["<s> " + i + "</s>" for i in token]

train = token[:int(len(token)*0.8)]
test= token[int(len(token)*0.8):]
# test

drive  sample_data
/content/drive
'My Drive'  'Shared drives'
/content/drive/My Drive
/content/drive/My Drive/assigmment_nlp
austin.txt  speeches.txt


['<s> so  no  that s not true </s>',
 '<s> well  putin was interviewed and i was interviewed last night </s>',
 '<s> i thought his interview   i though charlie rose did an excellent job </s>',
 '<s> but i thought his was a softer interview </s>',
 '<s> scott pelley interviewed me </s>',
 '<s> i thought scott was terrific </s>',
 '<s> i mean  it was a tough interview but i thought he was very fair and i thought the piece was very good </s>',
 '<s> why is he harder on me than putin </s>',
 '<s> i would say because putin is a nicer person than i am </s>',
 '<s> the worst in 36 years </s>',
 '<s> well  it s a good question and you can connect the dots </s>',
 '<s> but i guess it has been probably since  04 </s>',
 '<s> you know  you re talking about 10  11  12 years that people have wanted to bring the money back into the country </s>',
 '<s> and again it s called corporate inversion and in a lot of time corporations are moving out and they re going to other countries  taking jobs </s>',
 

#Getting n-Grams

In [4]:

def get_grams(n,token):
  token_split=[i.split(" ") for i in token]
  # print(token_split)

  arr=[]
  for i in token_split:
    for j in range(n-1,len(i)):
      t=i[j-(n-1):j+1]
      arr.append(" ".join(t))
  print(arr)

  joint_fdist = nltk.FreqDist(arr)
  # joint_fdist
  return joint_fdist


get_bi_grams = get_grams(2,token)
get_uni_grams = get_grams(1,token)
get_tri_grams = get_grams(3,token)
get_4_grams = get_grams(4,token)
# get_bi_grams


['<s> ', ' speech', 'speech 1', '1 ', ' ', ' ', ' ', ' ', ' thank', 'thank you', 'you so', 'so much', 'much </s>', '<s> that', 'that s', 's so', 'so nice', 'nice </s>', '<s> isn', 'isn t', 't he', 'he a', 'a great', 'great guy', 'guy </s>', '<s> he', 'he doesn', 'doesn t', 't get', 'get a', 'a fair', 'fair press', 'press ', ' he', 'he doesn', 'doesn t', 't get', 'get it', 'it </s>', '<s> it', 'it s', 's just', 'just not', 'not fair', 'fair </s>', '<s> and', 'and i', 'i have', 'have to', 'to tell', 'tell you', 'you i', 'i m', 'm here', 'here ', ' and', 'and very', 'very strongly', 'strongly here', 'here ', ' because', 'because i', 'i have', 'have great', 'great respect', 'respect for', 'for steve', 'steve king', 'king and', 'and have', 'have great', 'great respect', 'respect likewise', 'likewise for', 'for citizens', 'citizens united', 'united ', ' david', 'david and', 'and everybody', 'everybody ', ' and', 'and tremendous', 'tremendous resect', 'resect for', 'for the', 'the tea', 'tea 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



#n-gram probabilities

In [0]:
def n_gram_probl(get_grams_dict,n):
  ngram_probl={}
  for i in get_grams_dict.keys():

    k=i.split(" ")
    if(n==1):
      ngram_probl[i] = get_grams_dict[i]/len(get_uni_grams)
    elif(n==2):
      ngram_probl[i] = get_grams_dict[i]/get_uni_grams[k[0]]
    elif(n==3):
        ngram_probl[i] = get_grams_dict[i]/get_bi_grams[' '.join(k[:2])]
    elif(n==4):
      ngram_probl[i] = get_grams_dict[i]/get_tri_grams[' '.join(k[:3])]
    else:
      print("Enter a n value from {1,2,3,4}")

    
  return ngram_probl

#Generate Sentences

In [6]:
def generate(gram_probl):
  nis =list(gram_probl.keys())
  sent = []
  vals = np.array([gram_probl[i] for i in nis])
  vals = vals/vals.sum()

  while(1):
    probas = np.random.multinomial(100, vals, 1)
    idx = np.argmax(probas)
    newWord = nis[idx]

    if("<s>" in newWord):
      sent.append(newWord)
      break
  #     print(newWord)
      
  while(1):
    probas = np.random.multinomial(100, vals, 1)
    idx = np.argmax(probas)
    newWord = nis[idx]

    if("</s>" in newWord):
      sent.append(newWord)
  #     print(newWord)
      break
    else:
      sent.append(newWord)

  print(" ".join(sent))

# unigram_probl = n_gram_probl(get_uni_grams,1)
# print("Sentences generated by Uni-gram")
# generate(unigram_probl)

bigram_probl = n_gram_probl(get_bi_grams,2)
print("\nSentences generated by Bi-gram")
generate(bigram_probl)

trigram_probl = n_gram_probl(get_tri_grams,3)
print("\nSentences generated by Tri-gram")
generate(trigram_probl)

quad_gram_probl = n_gram_probl(get_4_grams,3)
print("\nSentences generated by Quad-gram")
generate(quad_gram_probl)


Sentences generated by Bi-gram
<s> they action </s>

Sentences generated by Tri-gram
<s> politicians are must have confidence with absolute murder to cut social  also the to repeal and coming into our ve beaten so can tell you position where that rebuild quickly our which could be say chuck   lies  they wouldn t education locally </s>

Sentences generated by Quad-gram
<s> irs  the s constant  it  reducing the budget numbers of deals to and fill that void iraq to egypt to political office  any  indeed  become i been saying for president get away with united  david and build  i build over time on planes just walking right in from massachusetts  that tremendous resect for the think of it  a better place </s>


In [18]:
def perplexity_calc(test_data,ngram_dick,n): 
  token_split=[i.split(" ") for i in test_data]
  perplexity=0
  for sent in token_split:
    x=0
    for i in range(n-1,len(sent)):

      nwords=sent[i-n+1:i+1]
      # print(nwords)
      if(ngram_dick[' '.join(nwords)]):
        x+=math.log(ngram_dick[' '.join(nwords)])

    perplexity+=x

  final_perplex=-1*perplexity/len(test_data)
  return final_perplex

print("Perplexity of bi gram is :",perplexity_calc(test,bigram_probl,2))
print("Perplexity of tri gram is :",perplexity_calc(test,trigram_probl,3))

Perplexity of bi gram is : 41.67798776298506
Perplexity of tri gram is : 20.556382320082836


###Readability of trigram & quadgram >> Readability of Uni and Bi gram ###

###Perplexity of Tri and Quad < Perplexity of bi and uni###